---
title: Calculating Br rat haplotype probabilities from HS founders
date: 11/8/2023
author: Sabrina
---

I am using Dan Munro's scripts to compute probabilities across the 8 founders per locus per individual. His code uses the R qtl2 package.

Github: [https://github.com/daniel-munro/qtl2-founder-haps/tree/main](https://github.com/daniel-munro/qtl2-founder-haps/tree/main)

The `genetic_map` subdirectory contains genetic mapping files from the older build rn6. I downloaded rn7 genetic markers from his preprint, [A revamped rat reference genome improves the discovery of genetic diversity in laboratory rats](https://www.biorxiv.org/content/10.1101/2023.04.13.536694v2.article-info), [Supplementary Table S2](https://www.biorxiv.org/content/biorxiv/early/2023/09/29/2023.04.13.536694/DC2/embed/media-2.xlsx?download=true).

I first saved Table S2 as a CSV, then split it by chromosome.

In [1]:
import pandas as pd
import gzip

# Read the original CSV file
df = pd.read_csv('/Users/sabrinami/Desktop/MAP.csv')

# Drop the 'CONTIG' column
# df = df.drop(columns=['CONTIG'])

,CONTIG,POS,CHR,cM
0,chr1,257394712,1,0.0000
1,chr1,257401401,1,0.0000
2,chr1,257385252,1,0.4775
3,chr1,257412479,1,0.6100
4,chr1,257366293,1,0.6100
...,...,...,...,...
150830,chr20,51679712,20,66.0760
150831,chr20,51749221,20,66.0760
150832,chr20,51731733,20,66.0760
150833,chr20,51651864,20,66.0760
